In [1]:
import pandas as pd
import os
import json

In [2]:
pwd = os.getcwd() 
filepath = os.path.join(pwd, 'Data', 'Data - Airbus Airlines.csv')
filepath1 = os.path.join(pwd, 'Data', 'Data - Boeing Airlines.csv')
filepath2 = os.path.join(pwd, 'Data', 'Data - Boeing_parts.csv') 
filepath3 = os.path.join(pwd, 'Data', 'Data - Airbus_parts.csv')
filepathjson = os.path.join(pwd, 'Json', 'Json - Aircraft Systems.json')
dataset_airbus_airlines = pd.read_csv(filepath, encoding='latin1')
dataset_boeing_airlines = pd.read_csv(filepath1, encoding='latin1')
dataset_boeing_parts = pd.read_csv(filepath2)
dataset_airbus_parts = pd.read_csv(filepath3)
dataset_replacement_interval = pd.read_json(filepathjson)


In [3]:
dataset_boeing_parts.info() # 2   Model Series  3398 non-null   object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1373 entries, 0 to 1372
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Part Number     1373 non-null   object
 1   Description     1373 non-null   object
 2   Aircraft Model  1373 non-null   object
dtypes: object(3)
memory usage: 32.3+ KB


boeing data manipulation

In [4]:
dataset_boeing_parts.sample(2)

,Part Number,Description,Aircraft Model
613,116A9110-27,panel assy - lwr lh,boeing 737-700
73,3202446-1,valve - check condit,boeing 737-800


Spliting the numbers in aircraft model

In [5]:
dataset_boeing_parts['Aircraft Model'] = dataset_boeing_parts['Aircraft Model'].str.split('boeing ',n=0,expand=True)[1]
dataset_boeing_parts

,Part Number,Description,Aircraft Model
0,4551,lamp - 28 volt 250 w,737-800
1,172A6123-1,panel assy - trailin,737-700
2,256A3310-6,transmission assy -,737-700
3,65C27384-26,duct assy,737-700
4,286A1082-005,wire bundle assy,737-700
...,...,...,...
1368,6273-5,fuse assy,737-300
1369,110A0113-19,door assy - #10 lh 5,737-700
1370,276A3100-9,reservoir assy,737-800
1371,07323P000-01,ballscrew assy,737-700


In [6]:
dataset_boeing_parts.sample()

,Part Number,Description,Aircraft Model
964,217A1117-1,duct assy,737-700


Adding aircraft system to the boeing part number database

In [7]:

# Load your aircraft systems JSON file
with open(filepathjson, 'r') as file:
    aircraft_systems = json.load(file)

def categorize_part(description):
    description = str(description).lower()
    
    for category, data in aircraft_systems.items():
        if any(common_part.lower() in description for common_part in data.get('common_parts', [])):
            return category
        if any(keyword.lower() in description for keyword in data.get('keywords', [])):
            return category
    return 'other'


def get_risk_level(description):
    category = categorize_part(description)
    return aircraft_systems.get(category, {}).get('risk_level', 'Unknown')

def get_priority(description):
    category = categorize_part(description)
    return aircraft_systems.get(category, {}).get('priority ', 'Unknown')


def get_primary_replacement_interval(description):
    category = categorize_part(description)
    replacement_dict = aircraft_systems.get(category, {}).get('replacement_interval', {})
    return list(replacement_dict.values())[0] if replacement_dict else 'On-condition'

def get_replacement_name_by_interval(description):
    category = categorize_part(description)
    interval = get_primary_replacement_interval(description)
    replacement_dict = aircraft_systems.get(category, {}).get('replacement_interval', {})
    for component, rep_interval in replacement_dict.items():
        if rep_interval == interval:
            return component
    return 'Unknown'

def get_geo_targets(description):
    category = categorize_part(description)
    return aircraft_systems.get(category, {}).get('geo_targets', [])

def get_countries(description):
    category = categorize_part(description)
    return aircraft_systems.get(category, {}).get('countries', [])

# Assuming your dataset is a pandas DataFrame with a 'Description' column
dataset_boeing_parts['Airsystem Category'] = dataset_boeing_parts['Description'].apply(categorize_part)
dataset_boeing_parts['Risk Level'] = dataset_boeing_parts['Description'].apply(get_risk_level)
dataset_boeing_parts['Priority'] = dataset_boeing_parts['Description'].apply(get_priority)
dataset_boeing_parts['Replacement Interval'] = dataset_boeing_parts['Description'].apply(get_primary_replacement_interval)
dataset_boeing_parts['Replacement Name'] = dataset_boeing_parts['Description'].apply(get_replacement_name_by_interval)
dataset_boeing_parts['Geo Location'] = dataset_boeing_parts['Description'].apply(get_geo_targets)    # <-- returns a list
dataset_boeing_parts['Countries'] = dataset_boeing_parts['Description'].apply(get_countries)        # <-- returns a list


In [8]:
dataset_boeing_parts.sample(10)

,Part Number,Description,Aircraft Model,Airsystem Category,Risk Level,Priority,Replacement Interval,Replacement Name,Geo Location,Countries
1320,352A1200-42,strut assy,737-700,landing_gear_system,High,Critical,Every 300-500 landings or on damage,Tires,"[Desert, Coastal, Urban, Mountain]","[Morocco, United States, Egypt, Canada, Germany]"
224,313A2121-8,panel assy,737-700,other,Unknown,Unknown,On-condition,Unknown,[],[]
203,5002R,inclinometer roll,737-700,other,Unknown,Unknown,On-condition,Unknown,[],[]
723,381800-1005,actuator assy - le s,737-800,landing_gear_system,High,Critical,Every 300-500 landings or on damage,Tires,"[Desert, Coastal, Urban, Mountain]","[Morocco, United States, Egypt, Canada, Germany]"
184,1173T211,switch - overheat zo,737-800,electrical_system,Moderate,Critical,Every 2 years or when capacity drops below 80%,Batteries,"[Urban, Island, Coastal, Desert, Mountain]","[Morocco, United States, India, France, Brazil..."
121,175A1704-13,panel assy - skin 34,737-700,other,Unknown,Unknown,On-condition,Unknown,[],[]
1195,2790475-103,valve assy - pressur,737-700,hydraulic_system,High,Critical,Every 2 years or on leak detection,Seals,"[Coastal, Desert, Tropical, Mountain]","[Morocco, India, Brazil, United States, Russia]"
502,273A2310-1,cylinder assy - mlg,737-800,hydraulic_system,High,Critical,Every 2 years or on leak detection,Seals,"[Coastal, Desert, Tropical, Mountain]","[Morocco, India, Brazil, United States, Russia]"
313,1A226-0217,track,737-800,cargo_handling_system,Moderate,Moderate,On-condition; inspect every 12 months,Motors,"[Continental, Desert, Urban]","[Qatar, China, Germany, USA, UAE]"
805,15801-040,control unit assy -,737-700,environmental_control_system,Moderate,Moderate,Every 1000 flight hours,Filters,"[Tropical, Desert, Coastal, Urban]","[Morocco, France, India, Australia, Brazil]"


In [11]:
dataset_boeing_parts = dataset_boeing_parts.explode(column='Countries')
dataset_boeing_parts = dataset_boeing_parts[dataset_boeing_parts['Airsystem Category'] != 'other']
dataset_boeing_parts.sample(10)

,Part Number,Description,Aircraft Model,Airsystem Category,Risk Level,Priority,Replacement Interval,Replacement Name,Geo Location,Countries
110,251A4430-4,switch assy - column,737-700,electrical_system,Moderate,Critical,Every 2 years or when capacity drops below 80%,Batteries,"[Urban, Island, Coastal, Desert, Mountain]",Japan
511,149A7558-14,panel - flap track a,737-700,flight_control_system,Critical,Critical,Every 5 years or on inspection failure,Control Cables,"[Mountain, Coastal, Urban, Desert]",United States
1164,183A0103-14,elevator assy,737-700,flight_control_system,Critical,Critical,Every 5 years or on inspection failure,Control Cables,"[Mountain, Coastal, Urban, Desert]",Morocco
4,286A1082-005,wire bundle assy,737-700,electrical_system,Moderate,Critical,Every 2 years or when capacity drops below 80%,Batteries,"[Urban, Island, Coastal, Desert, Mountain]",Brazil
924,606515-1,valve - check,737-700,hydraulic_system,High,Critical,Every 2 years or on leak detection,Seals,"[Coastal, Desert, Tropical, Mountain]",Brazil
1184,214C40-1-77,switch - barometric,737-700,electrical_system,Moderate,Critical,Every 2 years or when capacity drops below 80%,Batteries,"[Urban, Island, Coastal, Desert, Mountain]",Brazil
564,1024-3,switch - dimmer,737-800,electrical_system,Moderate,Critical,Every 2 years or when capacity drops below 80%,Batteries,"[Urban, Island, Coastal, Desert, Mountain]",India
272,7583695-101,filter assy,737-800,hydraulic_system,High,Critical,Every 2 years or on leak detection,Seals,"[Coastal, Desert, Tropical, Mountain]",Morocco
466,9000421-2,sensor - flap skew d,737-700,avionics_system,Moderate,High,On-condition or every 5 years,Displays,"[Urban, Island, Coastal, Mountain]",Germany
1173,213A3220-1,duct assy - heat exc,737-700,environmental_control_system,Moderate,Moderate,Every 1000 flight hours,Filters,"[Tropical, Desert, Coastal, Urban]",India


modifie the countries (look to notes/country modified)

In [12]:
countries = {
        'Brunei Darussalam' : 'Brunei',
       'Congo, the Democratic Republic' : 'Democratic Republic of the Congo',
       'Iran, Islamic Republic of' : 'Iran',
       'Slovakia (Slovak Republic)' : 'Slovakia',
       'State of Libya' : 'Libya',
       'Tanzania, United Republic of' : 'Tanzania',
       'Unidentified' : 'Unknown Country',
       'USA' : 'United States'
}
dataset_boeing_airlines.replace({'Country': countries}, inplace=True)

Joining boeing part with boeing airlines and replcement intreval

In [13]:
dataset_boeing_merged1 = dataset_boeing_parts.merge(dataset_boeing_airlines, how='left', left_on='Aircraft Model', right_on='Model Series')
dataset_boeing_merged1.sample(10)



,Part Number,Description,Aircraft Model,Airsystem Category,Risk Level,Priority,Replacement Interval,Replacement Name,Geo Location,Countries,Country,Airlines,Model Series,Region
489175,2S2072,valve - swing check,737-700,hydraulic_system,High,Critical,Every 2 years or on leak detection,Seals,"[Coastal, Desert, Tropical, Mountain]",United States,Morocco,Royal Air Maroc,737-700,Africa
1072998,315A2295-653,fan duct cowl and th,737-800,environmental_control_system,Moderate,Moderate,Every 1000 flight hours,Filters,"[Tropical, Desert, Coastal, Urban]",France,United States,American Airlines,737-800,North America
478390,548388-1,sensor,737-700,avionics_system,Moderate,High,On-condition or every 5 years,Displays,"[Urban, Island, Coastal, Mountain]",Morocco,Kenya,Kenya Airways,737-700,Africa
593638,30100022-3,extinguisher assy -,737-800,fire_protection_system,Critical,Critical,Every 5 years or after any discharge,Extinguishers,"[Urban, Island, Tropical, Mountain]",United States,United States,NAS Aviation Services LLC,737-800,North America
1325184,213A3220-4,duct assy - heat exc,737-700,environmental_control_system,Moderate,Moderate,Every 1000 flight hours,Filters,"[Tropical, Desert, Coastal, Urban]",Brazil,United States,International Lease Finance Co,737-700,North America
1270885,3000-46-27,switch - thermal,737-700,electrical_system,Moderate,Critical,Every 2 years or when capacity drops below 80%,Batteries,"[Urban, Island, Coastal, Desert, Mountain]",Germany,United States,NAS Aviation Services LLC,737-700,North America
794353,213A1212-8,duct assy,737-800,environmental_control_system,Moderate,Moderate,Every 1000 flight hours,Filters,"[Tropical, Desert, Coastal, Urban]",Australia,United States,CIT Aerospace LLC,737-800,North America
120218,251A4430-4,switch assy - column,737-800,electrical_system,Moderate,Critical,Every 2 years or when capacity drops below 80%,Batteries,"[Urban, Island, Coastal, Desert, Mountain]",Brazil,United States,American Airlines,737-800,North America
408855,315A2295-220,fan duct cowl and th,737-700,environmental_control_system,Moderate,Moderate,Every 1000 flight hours,Filters,"[Tropical, Desert, Coastal, Urban]",Brazil,United States,NAS Aviation Services LLC,737-700,North America
1127221,213A1201-76,duct assy,737-700,environmental_control_system,Moderate,Moderate,Every 1000 flight hours,Filters,"[Tropical, Desert, Coastal, Urban]",India,United States,Southwest Airlines,737-700,North America


rename the columns & export the data

In [16]:
# Remove columns : Countries Country Model Series Region
dataset_boeing_merged1 = dataset_boeing_merged1[['Part Number', 'Description', 'Aircraft Model', 'Airlines', 'Airsystem Category',
       'Risk Level', 'Priority', 'Replacement Interval', 'Replacement Name',
       'Geo Location']]
resultpath = os.path.join(pwd, 'Output', 'Data - Boeing.csv')
dataset_boeing_merged1.to_csv(resultpath, index=False )

In [17]:
dataset_boeing_merged1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1359268 entries, 0 to 1359267
Data columns (total 10 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   Part Number           1359268 non-null  object
 1   Description           1359268 non-null  object
 2   Aircraft Model        1359268 non-null  object
 3   Airlines              1359268 non-null  object
 4   Airsystem Category    1359268 non-null  object
 5   Risk Level            1359268 non-null  object
 6   Priority              1359268 non-null  object
 7   Replacement Interval  1359268 non-null  object
 8   Replacement Name      1359268 non-null  object
 9   Geo Location          1359268 non-null  object
dtypes: object(10)
memory usage: 103.7+ MB


removing the boeing 747-400 

In [ ]:
# dataset_boeing_parts['Aircraft Model'].unique()
# data = dataset_boeing_parts[dataset_boeing_parts['Aircraft Model'] != 'boeing 747-400']
# data.to_csv(os.path.join(pwd, 'Data', 'Data - Boeing_parts.csv'), index=False)

melting the data

In [ ]:
# ds = dataset.melt(id_vars=['airlines', 'Country', 'Region'],var_name='Model Type', value_name='Count') 
# ds[ds["Count"].isnull()] # Delete Opr Ord nan Del 
# # ds[ds["Model Type"].isnull()] # Replace .1 .2 .

# ds.to_csv(os.path.join(pwd, 'Data', 'Data - Airbus Airlines - Melted.csv'), index=False)
